In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import os
# cwd=os.getcwd()
# os.chdir('/content/drive/My Drive/Colab Notebooks/JantaHackJul25')

In [ ]:
import pandas as pd
train = pd.read_csv('/content/test_pFkWwen.csv')
test = pd.read_csv('/content/train_yaOffsB.csv')

Solution

Public leader board rank :

Private leader board rank :


### EDA

* We changed the ID column and added a date column starting from today's date.
* This was done to visualise the trends in each column and approaching this problem as a time series problem
* below you can find some snapshots we took by using tableau. It was quicker to visualise by using date.


The test/training datasets were combined first of all and were ordered using their ID. The very first trend encountered, was the repetitive pattern of estimated_insect_count

* We require a column which would change as soon as strictly increasing function drops. This will help model learn better .

* Insect counts Rises upto a value of 4000 then decreases.

![alt text](https://drive.google.com/uc?export=view&id=16PXTCN9EuoHosxxaFQSXCN8iSCyw-H_b)





A very similar pattern was found within the number_of_weeks_Quit.
* Rises upto to around 45 and then drops suddenly.

![alt text](https://drive.google.com/uc?export=view&id=1yCKahMHfQnfoKdacD6b6U2Zkl2obG1ih)

Again, the same pattern within number_of_weeks_used

![alt text](https://drive.google.com/uc?export=view&id=1QJx7JcWYBNEoy_0Tx6MlIrSF10ZqowqY)

Similarly, soil_type also pertained this increasing/decreasing trend. (Pattern)

![alt text](https://drive.google.com/uc?export=view&id=1nQohtXYU61Q20_A4Qol-p_O0BkrkiXeP)

From all these trends, we aimed at creating manual features (as raw features could get us to a maximum of 84% acc)

The features were, (with description)
1. Group -Group based on increasing sequence of Estimated_Insects_Count and Number_Doses_Week
2. Group_Change - Binary value indicating if the observation is the first observation of a new group in the same batch
3. Group_First - Binary value indicating if the observation is the first observation of the group
4. Group_Last - Binary value indicating if the observation is the last observation of the group
5. Batch_Last - Batch's Last Value


## Data Pre-Processing
* imputing values using -1 . Other imputations didnt improve the score

In [ ]:
# imputing all the null values with -1
train.fillna(-1, inplace=True)
test.fillna(-1,inplace=True)

In [ ]:
# logic behind ID number extraction, just delete the damn F
#train['ID'][0]
s = train['ID'][0]
print(s[1:])

00000002


In [ ]:
train1 = train
test1 = test

In [ ]:
train1['ID'] = train.apply(lambda x : int(x['ID'][1:]), axis = 1)
test1['ID'] = test.apply(lambda x : int(x['ID'][1:]), axis =1)

In [ ]:
#Appending both the datasets to create a combined dataset
result = train1.append(test1, sort=False)

In [ ]:
result = result.reset_index(drop=True)

In [ ]:
# sort by ID to find out patterns 
result= result.sort_values(by=['ID'])

## Feature Engineering

In [ ]:
# make the loop
# the features that we are targeting are - 
'''
1 - Group
2 - Group_Change 
3 - Group_First
4 - Group_Last
5 - Batch_Last
'''

# Initialization
result['Group'] = 0 
result['Group'][0] = 1
result['Group_Change'] = 0
result['Group_First'] = 0
result['Group_Last'] = 0
result['Batch_Last'] = 0
result['Group_First'][0] = 1
result['Soil_Change'] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
result

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,Group,Group_Change,Group_First,Group_Last,Batch_Last,Soil_Change
59310,1,188,1,0,1,0,0.0,0,1,0.0,0,0,0,0,0,0
0,2,188,1,1,1,0,-1.0,0,2,NaN,1,0,1,0,0,0
59311,3,209,1,0,1,0,0.0,0,2,1.0,0,0,0,0,0,0
59312,4,257,1,0,1,0,0.0,0,2,1.0,0,0,0,0,0,0
59313,5,257,1,1,1,0,0.0,0,2,1.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59307,155941,3702,1,0,2,10,-1.0,48,1,NaN,0,0,0,0,0,0
148166,155942,3702,1,0,2,10,25.0,18,3,0.0,0,0,0,0,0,0
59308,155943,3702,1,0,2,10,28.0,17,2,NaN,0,0,0,0,0,0
59309,155944,3895,1,0,2,5,52.0,7,1,NaN,0,0,0,0,0,0


In [ ]:
#Start loop
for i in range(2,result.shape[0]):
  if (abs(result['Estimated_Insects_Count'][i] - result['Estimated_Insects_Count'][i-1]) > 1):
    result['Group'][i] = result['Group'][i-1] + 1
    result['Group_First'][i] = 1
    result['Group_Last'][i-1] = 1
    result['Batch_Last'][i-1] = 1
    result['Group_Num'] = 1
  else:
    if (result['Number_Doses_Week'][i] >= result['Number_Doses_Week'][i-1]):
      if (result['Soil_Type'][i] == result['Soil_Type'][i-1]):
        result['Group'][i] = result['Group'][i-1]
        result['Group_Num'][i] = result['Group_Num'][i-1] + 1
      else:
        result['Group'][i] = result['Group'][i-1] + 1
        result['Group_Num'] = 1
    else:
      result['Group'][i] = result['Group'][i-1] + 1
      result['Group_Change'][i] = 1
      result['Group_First'][i] = 1
      result['Group_Last'][i-1] = 1
      result['Group'][i] = result['Group'][i-1] + 1
  # if (i%1000==0):
  #   # print(i,'th iteration completed')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: Se

In [ ]:
for i in range(1,(result.shape[0]-1)):
  if(result['Group_Last'][i] == 1 and result['Soil_Type'][i] != result['Soil_Type'][i+1]):
    result['Soil_Change'][i] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
result

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,Group,Group_Change,Group_First,Group_Last,Batch_Last,Soil_Change,Group_Num
59310,1,188,1,0,1,0,0.0,0,1,0.0,15718,0,1,1,1,0,1
0,2,188,1,1,1,0,-1.0,0,2,NaN,1,0,1,0,0,0,1
59311,3,209,1,0,1,0,0.0,0,2,1.0,15719,0,1,1,1,0,1
59312,4,257,1,0,1,0,0.0,0,2,1.0,15720,0,1,0,0,0,1
59313,5,257,1,1,1,0,0.0,0,2,1.0,15721,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59307,155941,3702,1,0,2,10,-1.0,48,1,NaN,15716,0,1,0,0,0,1
148166,155942,3702,1,0,2,10,25.0,18,3,0.0,34925,0,1,1,1,0,1
59308,155943,3702,1,0,2,10,28.0,17,2,NaN,15716,0,0,1,1,0,1
59309,155944,3895,1,0,2,5,52.0,7,1,NaN,15717,0,1,1,1,0,1


In [ ]:
testRedifined = result.loc[result['Crop_Damage'].isnull()]
trainRedifined = result.dropna()

In [ ]:
testRedifined = testRedifined.drop(columns=['ID','Crop_Damage']).reset_index(drop=True)
trainRedifined = trainRedifined.drop(columns=['ID']).reset_index(drop=True)

In [ ]:
trainRedifined

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,Group,Group_Change,Group_First,Group_Last,Batch_Last,Soil_Change,Group_Num
0,188,1,0,1,0,0.0,0,1,0.0,15718,0,1,1,1,0,1
1,209,1,0,1,0,0.0,0,2,1.0,15719,0,1,1,1,0,1
2,257,1,0,1,0,0.0,0,2,1.0,15720,0,1,0,0,0,1
3,257,1,1,1,0,0.0,0,2,1.0,15721,0,0,1,1,1,1
4,342,1,0,1,0,0.0,0,2,1.0,15722,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88853,3337,1,0,2,10,12.0,44,3,0.0,34923,0,1,1,1,0,1
88854,3516,1,0,2,10,20.0,38,1,0.0,34924,0,1,0,0,0,1
88855,3516,1,0,2,15,40.0,8,2,0.0,34924,0,0,1,1,0,1
88856,3702,1,0,2,10,25.0,18,3,0.0,34925,0,1,1,1,0,1


## Data Scaling

In [ ]:
# Robust Scalar
from sklearn.preprocessing import RobustScaler
train = trainRedifined.copy()

In [ ]:
trainRedifined

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,Group,Group_Change,Group_First,Group_Last,Batch_Last,Soil_Change,Group_Num
0,188,1,0,1,0,0.0,0,1,0.0,15718,0,1,1,1,0,1
1,209,1,0,1,0,0.0,0,2,1.0,15719,0,1,1,1,0,1
2,257,1,0,1,0,0.0,0,2,1.0,15720,0,1,0,0,0,1
3,257,1,1,1,0,0.0,0,2,1.0,15721,0,0,1,1,1,1
4,342,1,0,1,0,0.0,0,2,1.0,15722,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88853,3337,1,0,2,10,12.0,44,3,0.0,34923,0,1,1,1,0,1
88854,3516,1,0,2,10,20.0,38,1,0.0,34924,0,1,0,0,0,1
88855,3516,1,0,2,15,40.0,8,2,0.0,34924,0,0,1,1,0,1
88856,3702,1,0,2,10,25.0,18,3,0.0,34925,0,1,1,1,0,1


In [ ]:
train

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,Group,Group_Change,Group_First,Group_Last,Batch_Last,Soil_Change,Group_Num
0,188,1,0,1,0,0.0,0,1,0.0,15718,0,1,1,1,0,1
1,209,1,0,1,0,0.0,0,2,1.0,15719,0,1,1,1,0,1
2,257,1,0,1,0,0.0,0,2,1.0,15720,0,1,0,0,0,1
3,257,1,1,1,0,0.0,0,2,1.0,15721,0,0,1,1,1,1
4,342,1,0,1,0,0.0,0,2,1.0,15722,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88853,3337,1,0,2,10,12.0,44,3,0.0,34923,0,1,1,1,0,1
88854,3516,1,0,2,10,20.0,38,1,0.0,34924,0,1,0,0,0,1
88855,3516,1,0,2,15,40.0,8,2,0.0,34924,0,0,1,1,0,1
88856,3702,1,0,2,10,25.0,18,3,0.0,34925,0,1,1,1,0,1


In [ ]:
x_columns = train.columns

In [ ]:
list(x_columns[0:8])

['Estimated_Insects_Count',
 'Crop_Type',
 'Soil_Type',
 'Pesticide_Use_Category',
 'Number_Doses_Week',
 'Number_Weeks_Used',
 'Number_Weeks_Quit',
 'Season']

In [ ]:
scaler = RobustScaler()

train[list(x_columns[0:8])] = scaler.fit_transform(train[list(x_columns[0:8])])

train.head()

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,Group,Group_Change,Group_First,Group_Last,Batch_Last,Soil_Change,Group_Num
0,-0.877464,1.0,0.0,-1.0,-0.8,-1.3,-0.4375,-1.0,0.0,15718,0,1,1,1,0,1
1,-0.859469,1.0,0.0,-1.0,-0.8,-1.3,-0.4375,0.0,1.0,15719,0,1,1,1,0,1
2,-0.818338,1.0,0.0,-1.0,-0.8,-1.3,-0.4375,0.0,1.0,15720,0,1,0,0,0,1
3,-0.818338,1.0,1.0,-1.0,-0.8,-1.3,-0.4375,0.0,1.0,15721,0,0,1,1,1,1
4,-0.745501,1.0,0.0,-1.0,-0.8,-1.3,-0.4375,0.0,1.0,15722,0,1,1,1,1,1


In [ ]:
train.shape

(88858, 16)

Applying the same transformation in test as well

In [ ]:
test=testRedifined.copy()
test.head(3)

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Group,Group_Change,Group_First,Group_Last,Batch_Last,Soil_Change,Group_Num
0,188,1,1,1,0,-1.0,0,2,1,0,1,0,0,0,1
1,410,1,1,1,0,0.0,0,2,0,0,0,1,1,1,1
2,626,1,0,1,0,0.0,0,2,1,0,1,1,1,0,1


In [ ]:
from sklearn.preprocessing import RobustScaler
x_columns_test = test.columns
scaler = RobustScaler()
test[list(x_columns_test[0:8])] = scaler.fit_transform(test[list(x_columns_test[0:8])])
test.head()

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Group,Group_Change,Group_First,Group_Last,Batch_Last,Soil_Change,Group_Num
0,-0.877464,1.0,1.0,-1.0,-0.8,-1.35,-0.4375,0.0,1,0,1,0,0,0,1
1,-0.687232,1.0,1.0,-1.0,-0.8,-1.30,-0.4375,0.0,0,0,0,1,1,1,1
2,-0.502142,1.0,0.0,-1.0,-0.8,-1.30,-0.4375,0.0,1,0,1,1,1,0,1


In [ ]:
train.head(3)

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,Group,Group_Change,Group_First,Group_Last,Batch_Last,Soil_Change,Group_Num
0,-0.877464,1.0,0.0,-1.0,-0.8,-1.3,-0.4375,-1.0,0.0,15718,0,1,1,1,0,1
1,-0.859469,1.0,0.0,-1.0,-0.8,-1.3,-0.4375,0.0,1.0,15719,0,1,1,1,0,1
2,-0.818338,1.0,0.0,-1.0,-0.8,-1.3,-0.4375,0.0,1.0,15720,0,1,0,0,0,1


## Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = train.drop(columns = ['Crop_Damage'])
y = train['Crop_Damage']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

## Model Prototyping

In [ ]:
# trying lgb models 

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, train_test_split

In [ ]:
# pd.DataFrame(X).isnull().sum()

X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.3,random_state=0)

# lg=LGBMClassifier(boosting_type='gbdt', max_depth=14, learning_rate=0.14, objective='multiclass', reg_alpha=0,
#                   reg_lambda=1, n_jobs=-1, random_state=100, n_estimators=600)

lg = LGBMClassifier()


lg.fit(X_train,Y_train)
Y_predict=lg.predict(X_test)

print(f1_score(Y_test,Y_predict, average='weighted'))

print(accuracy_score(Y_test,Y_predict))


0.905651457148903
0.9145847400405132


In [ ]:
X_main_test = test.copy()
X_main_test.shape

(59310, 15)

In [ ]:
X_main_test.head(3)

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Group,Group_Change,Group_First,Group_Last,Batch_Last,Soil_Change,Group_Num
0,-0.877464,1.0,1.0,-1.0,-0.8,-1.35,-0.4375,0.0,1,0,1,0,0,0,1
1,-0.687232,1.0,1.0,-1.0,-0.8,-1.30,-0.4375,0.0,0,0,0,1,1,1,1
2,-0.502142,1.0,0.0,-1.0,-0.8,-1.30,-0.4375,0.0,1,0,1,1,1,0,1


In [ ]:
test_org = pd.read_csv('test_pFkWwen.csv')
ID = test_org['ID']

In [ ]:

Y_test_predict=lg.predict(X_main_test)

sample_submission=pd.read_csv('/content/sample_submission_O1oDc4H.csv')
sample_submission['Crop_Damage']=Y_test_predict
sample_submission = sample_submission.drop(axis=1, columns=['ID'])
sample_submission['ID'] = ID

sample_submission.head(3)

,Crop_Damage,ID
0,1.0,F00000002
1,1.0,F00000007
2,1.0,F00000011


In [ ]:
sample_submission.to_csv('lgb_base_new1.csv', index=False)

K fold ensemble (metric :accuracy score)
* 10 fold with prediction
* finally mode is taken from all the predictions 

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np


In [ ]:
X=X.copy()
Y=y.copy()
X_main_test=pd.DataFrame(X_main_test)

kf=KFold(n_splits=10, random_state=100, shuffle=True)

y_test_predict=0
f1_score1 = 0
acc_score1 = 0
j=1
result={}
for i, (train_index, test_index) in enumerate(kf.split(train)):
    
   Y_train, Y_valid = Y.iloc[train_index], Y.iloc[test_index]
   X_train, X_valid = X.iloc[train_index,:], X.iloc[test_index,:]
   
   print( "\nFold ", j)
    
   model=lg.fit(X_train,Y_train)
   pred = model.predict(X_valid)
   
   print(f1_score(Y_valid,pred, average='weighted'))
   print(accuracy_score(Y_valid,pred))
   f1_score1+=f1_score(Y_valid,pred, average='weighted')
   acc_score1+=accuracy_score(Y_valid,pred)
  
   y_test_predict+=model.predict(X_main_test)  
   result[j]=model.predict(X_main_test)
   j+=1
   
    
results=y_test_predict/10   
results=np.round(results, decimals=0)
print(f1_score1/10)
print(acc_score1/10)

d = pd.DataFrame()
for i in range(1, 11):
    d = pd.concat([d,pd.DataFrame(result[i])],axis=1)
d.columns=['1','2','3','4','5','6','7','8','9','10']


Fold  1
0.9086780171335394
0.91739815440018

Fold  2
0.9030929832827188
0.911883862255233

Fold  3
0.9044334335619576
0.9131217645734864

Fold  4
0.9091519752397423
0.91739815440018

Fold  5
0.9066285004203355
0.9158226423587666

Fold  6
0.9095911801258113
0.9181859104208868

Fold  7
0.9069726242662981
0.9153724960612198

Fold  8
0.9046405663541626
0.9144722034661265

Fold  9
0.9034053279835937
0.9127743387732132

Fold  10
0.9083324920525441
0.9175014068655036
0.9064927100420703
0.9153930933574795


In [ ]:
re = d.mode(axis=1)[0]
sample_submission=pd.read_csv('/content/sample_submission_O1oDc4H.csv')
sample_submission = sample_submission.drop(axis=1, columns=['ID'])
sample_submission['ID'] = ID


sample_submission = sample_submission.drop(axis=1, columns=['Crop_Damage'])
sample_submission['Crop_Damage']=re
sample_submission.head(3)

,ID,Crop_Damage
0,F00000002,1.0
1,F00000007,1.0
2,F00000011,1.0


In [ ]:
sample_submission.to_csv('k_fold_lgb_latest.csv', index=False)

In [ ]:
test_org.head(3)

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season
0,F00000002,188,1,1,1,0,NaN,0,2
1,F00000007,410,1,1,1,0,0.0,0,2
2,F00000011,626,1,0,1,0,0.0,0,2


### Random Forest Model Training

In [ ]:
#Random Forest Approach
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model 
rf = RandomForestClassifier(n_estimators= 10, random_state=42)
# Train the model on training data
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
print('Accuracy:', round(accuracy, 2), '%.')
# THIS IS THE BASELINE MODEL

Accuracy: 0.91 %.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 42)

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 80, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 17, 24, 31, 38, 45, 52, 59, 66, 73, 80, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 288, 377, 466, 555, 644, 733, 822, 911, 1000]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier(random_state = 42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_train, y_train);

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored

In [ ]:
rf_random.best_params_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    accuracy = accuracy_score(test_labels,predictions)
    print('Model Performance')
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [ ]:
import joblib
import pickle
filename = 'best_random_feature_ENGINEERING.sav'
joblib.dump(best_random, filename)

### Second Model Training

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
import joblib
import pickle
filename = 'XGBOOST_IMPUTED.sav'
joblib.dump(best_xgboost_random, filename)